**Author:** Storm Pino  
**Team Role:** Climate Lead (Member A)  

Fetches Temperature and Precipitation Data from NOAA and parses it from 1985-2024 by month.

In [3]:
import pandas as pd
import requests
from io import StringIO
import time

state_map = {
    1: "Alabama", 2: "Arizona", 3: "Arkansas", 4: "California", 5: "Colorado",
    6: "Connecticut", 7: "Delaware", 8: "Florida", 9: "Georgia", 10: "Idaho",
    11: "Illinois", 12: "Indiana", 13: "Iowa", 14: "Kansas", 15: "Kentucky",
    16: "Louisiana", 17: "Maine", 18: "Maryland", 19: "Massachusetts", 20: "Michigan",
    21: "Minnesota", 22: "Mississippi", 23: "Missouri", 24: "Montana", 25: "Nebraska",
    26: "Nevada", 27: "New Hampshire", 28: "New Jersey", 29: "New Mexico", 30: "New York",
    31: "North Carolina", 32: "North Dakota", 33: "Ohio", 34: "Oklahoma", 35: "Oregon",
    36: "Pennsylvania", 37: "Rhode Island", 38: "South Carolina", 39: "South Dakota", 40: "Tennessee",
    41: "Texas", 42: "Utah", 43: "Vermont", 44: "Virginia", 45: "Washington",
    46: "West Virginia", 47: "Wisconsin", 48: "Wyoming", 49: "Alaska", 50: "Hawaii"
}

def fetch_cag_state_series(param="tavg", start=1985, end=2024):
    """Fetches NOAA Climate-at-a-Glance data for all states except Alaska."""
    all_states = []
    for sid, state_name in state_map.items():
        if state_name == "Alaska":
            print(f"Skipping {state_name} — no statewide data.")
            continue

        url = f"https://www.ncei.noaa.gov/access/monitoring/climate-at-a-glance/statewide/time-series/{sid}/{param}/1/0/{start}-{end}/data.csv"
        try:
            r = requests.get(url, timeout=20)
            r.raise_for_status()

            # NOAA files: 2 comment lines, then header
            df = pd.read_csv(StringIO(r.text), skiprows=2)
            if list(df.columns) != ["Date", "Value"]:
                raise ValueError(f"Unexpected columns: {df.columns.tolist()}")

            df = df.rename(columns={"Date": "date", "Value": param})
            df["State"] = state_name
            df["year"] = df["date"].astype(str).str[:4].astype(int)
            df["month"] = df["date"].astype(str).str[-2:].astype(int)
            all_states.append(df[["State", "year", "month", param]])

            print(f"{state_name:15} | {param} | {df.shape[0]} rows")
            time.sleep(0.4)

        except Exception as e:
            print(f"Skipped {state_name:15} | {param} — {e}")

    return pd.concat(all_states, ignore_index=True)

df_t = fetch_cag_state_series("tavg", 1985, 2024)

df_p = fetch_cag_state_series("pcp", 1985, 2024)

# Merge the two datasets
df_climate = pd.merge(df_t, df_p, on=["State", "year", "month"], how="inner")

print("\nFinal NOAA Climate Dataset:")
print("Shape:", df_climate.shape)
display(df_climate.head())

# Save to file
df_climate.to_csv("../data/processed/noaa_monthly_climate_1985_2024.csv", index=False)

Fetching Temperature (tavg)...
Alabama         | tavg | 480 rows
Arizona         | tavg | 480 rows
Arkansas        | tavg | 480 rows
California      | tavg | 480 rows
Colorado        | tavg | 480 rows
Connecticut     | tavg | 480 rows
Delaware        | tavg | 480 rows
Florida         | tavg | 480 rows
Georgia         | tavg | 480 rows
Idaho           | tavg | 480 rows
Illinois        | tavg | 480 rows
Indiana         | tavg | 480 rows
Iowa            | tavg | 480 rows
Kansas          | tavg | 480 rows
Kentucky        | tavg | 480 rows
Louisiana       | tavg | 480 rows
Maine           | tavg | 480 rows
Maryland        | tavg | 480 rows
Massachusetts   | tavg | 480 rows
Michigan        | tavg | 480 rows
Minnesota       | tavg | 480 rows
Mississippi     | tavg | 480 rows
Missouri        | tavg | 480 rows
Montana         | tavg | 480 rows
Nebraska        | tavg | 480 rows
Nevada          | tavg | 480 rows
New Hampshire   | tavg | 480 rows
New Jersey      | tavg | 480 rows
New Mexico      |

,State,year,month,tavg,pcp
0,Alabama,1985,1,37.2,3.90
1,Alabama,1985,2,45.5,6.43
2,Alabama,1985,3,58.7,2.49
3,Alabama,1985,4,64.0,2.93
4,Alabama,1985,5,70.6,4.11


Saved to: ../data/processed/noaa_monthly_climate_1985_2024.csv
